You have to work on the Dogs adoptions dataset.

In [78]:
import pandas as pd
import numpy as np
import re

Importing the data

In [79]:
dogs=pd.read_csv("dogs.csv", sep=",",engine='python')
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


In [80]:
dogs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58180 entries, 0 to 58179
Data columns (total 37 columns):
id                 58180 non-null int64
org_id             58180 non-null object
url                58180 non-null object
type.x             58180 non-null object
species            58180 non-null object
breed_primary      58180 non-null object
breed_secondary    20821 non-null object
breed_mixed        58180 non-null bool
breed_unknown      58180 non-null bool
color_primary      26134 non-null object
color_secondary    12059 non-null object
color_tertiary     1217 non-null object
age                58180 non-null object
sex                58180 non-null object
size               58180 non-null object
coat               27185 non-null object
fixed              58180 non-null bool
house_trained      58180 non-null bool
declawed           0 non-null float64
special_needs      58180 non-null bool
shots_current      58180 non-null bool
env_children       28027 non-null object
env_do

A description of the variables in the dataset

*id* = The unique PetFinder identification number for each animal.

*org_id* = The unique identification number for each shelter or rescue.

*url* = The URL for each animal’s listing.

*species* = Species of animal.

*breed_primary* = The primary (assumed) breed assigned by the shelter or rescue.

*breed_secondary* = The secondary (assumed) breed assigned by the shelter or rescue.

*breed_mixed* = Whether or not an animal is presumed to be mixed breed

*breed_unknown*	= Whether or not the animal’s breed is completely unknown.

*color_primary*= The most prevalent color of an animal.

*color_secondary*= The second most prevalent color of an animal.

*color_tertiary*= The third most prevalent color of an animal.

*age*= The assumed age class of an animal (Baby, Young, Adult, or Senior).

*sex*= The sex of an animal (Female, Male, or Unknown).

*size*= The general size class of an animal (Small, Medium, Large, Extra Large).

*coat*= Coat Length for each animal (Curly, Hairless, Long, Medium, Short, Wire)

*fixed*= Whether or not an animal has been spayed/neutered.

*house_trained*= Whether or not an animal is trained to not go to the bathroom in the house.

*declawed*= Whether or not the animal has had its dewclaws removed.

*special_needs*= Whether or not the animal is considered to have special needs (this can be a long-term medical condition or particular temperament that requires extra care).

*shots_current*= Whether or not the animal is up to date on all of their vaccines and other shots.

*env_children*= Whether or not the animal is recommended for a home with children.

*env_dogs*= Whether or not the animal is recommended for a home with other dogs.

*env_cats*= Whether or not the animal is recommended for a home with cats.

*name* =The animal’s name (as given by the shelter/rescue).

*tags*= Any tags given to the dog by the shelter rescue (pipe

*photo*=The URL to the animal’s primary photo.

*status*= Whether the animal is adoptable or not.

*posted*= The date that this animal was first listed on PetFinder .

*contact_city*= The rescue/shelter’s listed city.

*contact_state*= The rescue/shelter’s listed state.

*contact_zip* =	The rescue/shelter’s listed zip code.

*contact_country* = The rescue/shelter’s listed country.

*stateQ*= The state abbreviation queried in the API to return this result .

*accessed*=The date that this data was acquired from the PetFinder API.

*type*= The type of animal.

*description*= The full description of an animal, as entered by the rescue or shelter. This is the only field returned by the V1 API.

We can see that when status is not adoptable we have a dataset import problem. These rows all seem to be shifted by one column

In [81]:
dogs[dogs["status"]!="adoptable"][["status","posted","contact_city","contact_state","contact_zip","contact_country","stateQ","accessed"]]

,status,posted,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed
644,2018-04-05T05:18:31+0000,Las Vegas,NV,89146,US,89009,2019-09-20,NaN
5549,2017-05-26T21:43:16+0000,Chandler,AZ,85249,US,AZ,2019-09-20,NaN
10888,2019-09-01T15:12:06+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
11983,2019-08-06T12:15:58+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
12495,2019-07-18T14:20:58+0000,Albany,NY,12220,US,CT,2019-09-20,NaN
12600,2019-07-11T20:34:42+0000,Saugerties,NY,12477,US,CT,2019-09-20,NaN
12613,2019-07-11T14:16:38+0000,Saugerties,NY,12477,US,CT,2019-09-20,NaN
17619,2019-08-10T16:00:35+0000,Bristow,VA,20136,US,DC,2019-09-20,NaN
18611,2019-05-14T21:09:27+0000,Silver Spring,MD,20905,US,DC,2019-09-20,NaN
19747,2016-12-15T13:33:43+0000,Gettysburg,PA,17325,US,DC,2019-09-20,NaN


We can solve the problem with the following way

In [82]:
col=["status","posted","contact_city","contact_state","contact_zip","contact_country","stateQ","accessed"]
m=dogs["status"]!="adoptable"
dogs.loc[m,col]=dogs.loc[m,col].shift(axis=1)
dogs.loc[m,col]
dogs.loc[m,"status"]='not adoptable'

As a result, we can now correctly import the dates

In [83]:
dogs["accessed"]=pd.to_datetime(dogs["accessed"])
dogs["accessed"]

0       2019-09-20
1       2019-09-20
2       2019-09-20
3       2019-09-20
4       2019-09-20
           ...    
58175   2019-09-20
58176   2019-09-20
58177   2019-09-20
58178   2019-09-20
58179   2019-09-20
Name: accessed, Length: 58180, dtype: datetime64[ns]

In [84]:
dogs["posted"]=pd.to_datetime(dogs["posted"])
dogs["posted"]

0       2019-09-20 16:37:59+00:00
1       2019-09-20 16:24:57+00:00
2       2019-09-20 14:10:11+00:00
3       2019-09-20 10:08:22+00:00
4       2019-09-20 06:48:30+00:00
                   ...           
58175   2019-05-03 14:23:49+00:00
58176   2019-04-13 16:20:24+00:00
58177   2018-09-27 04:18:56+00:00
58178   2018-09-12 05:03:38+00:00
58179   2018-09-03 20:42:24+00:00
Name: posted, Length: 58180, dtype: datetime64[ns, UTC]

As we can see, stateQ should be an abbreviation instead for some lines we have a zip code

In [85]:
dogs.groupby("stateQ").count().head(12)

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,posted,contact_city,contact_state,contact_zip,contact_country,accessed,type.y,description,stay_duration,stay_cost
stateQ,,,,,,,,,,,,,,,,,,,,,
89009,747,747,747,747,747,747,257,747,747,380,...,747,747,747,747,747,747,745,606,747,747
89014,1,1,1,1,1,1,0,1,1,0,...,1,1,1,1,1,1,1,0,1,1
89024,284,284,284,284,284,284,75,284,284,48,...,284,284,284,284,284,284,266,257,284,284
89027,2,2,2,2,2,2,0,2,2,0,...,2,2,2,2,2,2,2,2,2,2
89121,25,25,25,25,25,25,6,25,25,5,...,25,25,25,25,25,25,25,5,25,25
89406,181,181,181,181,181,181,89,181,181,66,...,181,181,181,181,181,181,181,181,181,181
89408,8,8,8,8,8,8,2,8,8,8,...,8,8,8,8,8,8,8,8,8,8
89423,435,435,435,435,435,435,151,435,435,260,...,435,435,435,435,435,435,424,388,435,435
89431,37,37,37,37,37,37,32,37,37,1,...,37,37,37,37,37,37,37,1,37,37


Looking it up reveals that zip codes beginning with 89 refer to the state of nevada

In [86]:
dogs["stateQ"]=dogs["stateQ"].str.replace(r"\d+","NV")

In [87]:
dogs.groupby("stateQ").count()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,posted,contact_city,contact_state,contact_zip,contact_country,accessed,type.y,description,stay_duration,stay_cost
stateQ,,,,,,,,,,,,,,,,,,,,,
AK,15,15,15,15,15,15,4,15,15,15,...,15,15,15,15,15,15,15,15,15,15
AL,1043,1043,1043,1043,1043,1043,477,1043,1043,461,...,1043,1043,1043,1043,1043,1043,1040,715,1043,1043
AR,695,695,695,695,695,695,223,695,695,386,...,695,695,695,695,695,695,693,666,695,695
AZ,2170,2170,2170,2170,2170,2170,950,2170,2170,785,...,2170,2170,2170,2170,2170,2170,2157,1564,2170,2170
CA,1028,1028,1028,1028,1028,1028,500,1028,1028,274,...,1028,1028,1028,1028,1028,1028,1021,586,1028,1028
CO,1773,1773,1773,1773,1773,1773,518,1773,1773,395,...,1773,1773,1773,1773,1773,1773,1759,1637,1773,1773
CT,6730,6730,6730,6730,6730,6730,2487,6730,6730,3608,...,6730,6730,6730,6729,6730,6730,6659,6456,6730,6730
DC,4669,4669,4669,4669,4669,4669,1446,4669,4669,1851,...,4669,4669,4669,4669,4669,4669,4614,4335,4669,4669
DE,2367,2367,2367,2367,2367,2367,880,2367,2367,1215,...,2367,2367,2367,2367,2367,2367,2322,2207,2367,2367


Looking the dataset we see other strange zip code but we discover that they are different because they are canadian postal code

In [88]:
dogs.groupby("contact_zip").count().tail(4)

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,posted,contact_city,contact_state,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
contact_zip,,,,,,,,,,,,,,,,,,,,,
E7L 1Y8,2,2,2,2,2,2,1,2,2,0,...,2,2,2,2,2,2,2,2,2,2
G5X 1C2,2,2,2,2,2,2,1,2,2,1,...,2,2,2,2,2,2,2,2,2,2
J0B 3E0,10,10,10,10,10,10,2,10,10,0,...,10,10,10,10,10,10,10,10,10,10
J2K 1N4,2,2,2,2,2,2,0,2,2,0,...,2,2,2,2,2,2,2,1,2,2


Analisis of missing value: looking to the percentages of missing values for each rows

In [89]:
columns=list(dogs.columns)
missing={}
for elem in columns:
    missing[elem]=sum(dogs[elem].isnull())/len(dogs[elem])
missing

{'id': 0.0,
 'org_id': 0.0,
 'url': 0.0,
 'type.x': 0.0,
 'species': 0.0,
 'breed_primary': 0.0,
 'breed_secondary': 0.6421278789962186,
 'breed_mixed': 0.0,
 'breed_unknown': 0.0,
 'color_primary': 0.5508078377449295,
 'color_secondary': 0.7927294602956343,
 'color_tertiary': 0.9790821588174631,
 'age': 0.0,
 'sex': 0.0,
 'size': 0.0,
 'coat': 0.5327432107253351,
 'fixed': 0.0,
 'house_trained': 0.0,
 'declawed': 1.0,
 'special_needs': 0.0,
 'shots_current': 0.0,
 'env_children': 0.5182708834651083,
 'env_dogs': 0.40410794087315227,
 'env_cats': 0.667377105534548,
 'name': 0.0,
 'status': 0.0,
 'posted': 0.0,
 'contact_city': 0.0,
 'contact_state': 0.0,
 'contact_zip': 0.00020625644551392232,
 'contact_country': 0.0,
 'stateQ': 0.0,
 'accessed': 0.0,
 'type.y': 0.011000343760742524,
 'description': 0.14962186318322448,
 'stay_duration': 0.0,
 'stay_cost': 0.0}

64% of breed_secondary it is missing, how can we interpret this?

Analysis of the second dataset

In [90]:
travels=pd.read_csv("dogTravel.csv")
travels.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In [91]:
travels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6194 entries, 0 to 6193
Data columns (total 9 columns):
index            6194 non-null int64
id               6194 non-null int64
contact_city     6194 non-null object
contact_state    6194 non-null object
description      6194 non-null object
found            6194 non-null object
manual           2147 non-null object
remove           1738 non-null object
still_there      319 non-null object
dtypes: int64(2), object(7)
memory usage: 435.6+ KB


Description of the variables

*id* The unique PetFinder identification number for each animal

*contact_city*  The rescue/shelter’s listed city

*contact_state* The rescue/shelter’s listed State

*description* The full description of each animal as entered by the rescue/shelter

*found*  Where the animal was found. Note: this is a mixed bag of values States/countries/cities

*remove* Animal removed from location

*still_there* Whether the animal is still located in their origin location and will be transported to their final destination after adoption.

Analysis of missing data

In [92]:
travels.isnull().any()

index            False
id               False
contact_city     False
contact_state    False
description      False
found            False
manual            True
remove            True
still_there       True
dtype: bool

Third dataset

In [93]:
nst=pd.read_csv("NST-EST2021-POP.csv",header=None,names=["State","Population"])
nst.head()

,State,Population
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223


I remove the dots so I can convert to numerical

In [94]:
nst["Population"]=nst["Population"].str.replace(".","").astype("int")

1. Extract all dogs with status that is not adoptable

In [95]:
dogs[dogs["status"]!="adoptable"][["status","id"]]

,status,id
644,not adoptable,41330726
5549,not adoptable,38169117
10888,not adoptable,45833989
11983,not adoptable,45515547
12495,not adoptable,45294115
12600,not adoptable,45229004
12613,not adoptable,45227052
17619,not adoptable,45569380
18611,not adoptable,44694387
19747,not adoptable,36978896


2. For each (primary) breed, determine the number of dogs

In [96]:
totali=dogs.groupby("breed_primary",as_index=False).count()[["breed_primary","id"]]
totali

,breed_primary,id
0,Affenpinscher,17
1,Afghan Hound,4
2,Airedale Terrier,19
3,Akbash,3
4,Akita,181
...,...,...
211,Wirehaired Pointing Griffon,1
212,Wirehaired Terrier,60
213,Xoloitzcuintli / Mexican Hairless,11
214,Yellow Labrador Retriever,158


Problem of breed and missing data, What can we do?

For the following instances i have breed_primary as mixed but the attribute breed_mixed sometimes is true and sometimes is false, how can i interpret this?


In [97]:
dogs[dogs["breed_primary"]=="Mixed Breed"]

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
278,45821826,NV39,https://www.petfinder.com/dog/chance-45821826/...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Las Vegas,NV,89118,US,NV,2019-09-20,Dog,Greetings and salutations! My name is Chance a...,115,398.68
340,45700110,NV181,https://www.petfinder.com/dog/kiki-45700110/nv...,Dog,Dog,Mixed Breed,NaN,True,False,NaN,...,Las Vegas,NV,89147,US,NV,2019-09-20,Dog,Our very perfect Kiki is searching the city fo...,92,16.24
346,45667997,NV155,https://www.petfinder.com/dog/coco-2-45667997/...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Las Vegas,NV,89103,US,NV,2019-09-20,Dog,Meet CoCo 2\nShe is 14 pds 2 years old and it...,95,203.55
367,45583244,AZ662,https://www.petfinder.com/dog/ozzy-45583244/az...,Dog,Dog,Mixed Breed,NaN,True,False,White / Cream,...,Kingman,AZ,86401,US,NV,2019-09-20,Dog,"Ozzy is a beautiful special needs boy, he is a...",133,213.33
441,45221744,NV155,https://www.petfinder.com/dog/milo-45221744/nv...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Las Vegas,NV,89103,US,NV,2019-09-20,Dog,"Meet Milo he a sweetheart ,just have to give h...",85,356.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57972,43801611,WV50,https://www.petfinder.com/dog/butch-43801611/w...,Dog,Dog,Mixed Breed,NaN,True,False,NaN,...,Beckley,WV,25802,US,WV,2019-09-20,Dog,NaN,68,280.09
57984,43492553,WV206,https://www.petfinder.com/dog/zeke-43492553/wv...,Dog,Dog,Mixed Breed,NaN,True,False,NaN,...,Princeton,WV,24740,US,WV,2019-09-20,Dog,WEIGHT 30LBS NO CHIP MP,70,278.81
58001,42628397,WV143,https://www.petfinder.com/dog/ethel-jane-starr...,Dog,Dog,Mixed Breed,NaN,True,False,NaN,...,Fairmont,WV,26554,US,WV,2019-09-20,Dog,Please contact Pet (information@pethelpersinc....,108,304.34
58004,42602232,WV50,https://www.petfinder.com/dog/nakita-42602232/...,Dog,Dog,Mixed Breed,NaN,True,False,NaN,...,Beckley,WV,25802,US,WV,2019-09-20,Dog,NaN,119,271.55


Let's investigate this situation further

In [98]:
dogs[(dogs["breed_secondary"]=="Mixed Breed")& (dogs["breed_mixed"]==False)]

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost


In [99]:
dogs[(dogs["breed_primary"]=="Mixed Breed")& (dogs["breed_secondary"]=="Mixed Breed")]

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost


In [100]:
dogs[(dogs["breed_primary"]=="Mixed Breed")& (dogs["breed_mixed"]==False)]

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
278,45821826,NV39,https://www.petfinder.com/dog/chance-45821826/...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Las Vegas,NV,89118,US,NV,2019-09-20,Dog,Greetings and salutations! My name is Chance a...,115,398.68
346,45667997,NV155,https://www.petfinder.com/dog/coco-2-45667997/...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Las Vegas,NV,89103,US,NV,2019-09-20,Dog,Meet CoCo 2\nShe is 14 pds 2 years old and it...,95,203.55
441,45221744,NV155,https://www.petfinder.com/dog/milo-45221744/nv...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Las Vegas,NV,89103,US,NV,2019-09-20,Dog,"Meet Milo he a sweetheart ,just have to give h...",85,356.51
449,45169148,NV155,https://www.petfinder.com/dog/jainie-45169148/...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Las Vegas,NV,89103,US,NV,2019-09-20,Dog,"Janie is mixed breed about 7 pds 2 years old, ...",69,198.31
1075,46025130,CA754,https://www.petfinder.com/dog/martha-46025130/...,Dog,Dog,Mixed Breed,NaN,False,False,Black,...,Truckee,CA,96161,US,NV,2019-09-20,Dog,"Personality: People Oriented, Mellow & Sweet\n...",93,271.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57739,45606960,VA253,https://www.petfinder.com/dog/hashbrown-456069...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Roanoke,VA,24016,US,WV,2019-09-20,Dog,NaN,76,197.29
57740,45606956,VA253,https://www.petfinder.com/dog/starbright-45606...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Roanoke,VA,24016,US,WV,2019-09-20,Dog,NaN,98,290.30
57741,45606954,VA253,https://www.petfinder.com/dog/mulan-45606954/v...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Roanoke,VA,24016,US,WV,2019-09-20,Dog,NaN,107,262.01
57872,44895276,WV184,https://www.petfinder.com/dog/red-44895276/wv/...,Dog,Dog,Mixed Breed,NaN,False,False,Red / Chestnut / Orange,...,Marlinton,WV,24954,US,WV,2019-09-20,Dog,He is in need of rescue or adoption from the P...,74,251.39


3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed

We noticed that non Mixed breed dogs had nan value for attribute Secondary Breed

In [101]:
def mixbreed_ratio(x):
    if sum(pd.isna(x)) != 0:
        return sum(pd.notna(x)) / sum(pd.isna(x))
    else:
        return np.inf

dogs.groupby('breed_primary')['breed_secondary'].apply(mixbreed_ratio)

breed_primary
Affenpinscher                        0.133333
Afghan Hound                         0.333333
Airedale Terrier                     0.900000
Akbash                               0.000000
Akita                                0.403101
                                       ...   
Wirehaired Pointing Griffon          0.000000
Wirehaired Terrier                   0.428571
Xoloitzcuintli / Mexican Hairless    0.571429
Yellow Labrador Retriever            0.645833
Yorkshire Terrier                    0.558442
Name: breed_secondary, Length: 216, dtype: float64

4. For each (primary) breed, determine the earliest and the latest posted timestamp.

In [102]:
latest=dogs.groupby("breed_primary",as_index=False).max()[["breed_primary","posted"]]
earliest=dogs.groupby("breed_primary",as_index=False).min()[["breed_primary","posted"]]
pd.merge(earliest,latest,on="breed_primary",suffixes=['_earliest', '_latest'])

,breed_primary,posted_earliest,posted_latest
0,Affenpinscher,2012-03-08 10:27:33+00:00,2019-09-14 10:10:51+00:00
1,Afghan Hound,2017-06-29 23:28:51+00:00,2019-07-27 00:38:48+00:00
2,Airedale Terrier,2014-06-13 12:59:36+00:00,2019-09-19 18:40:39+00:00
3,Akbash,2019-07-21 00:35:59+00:00,2019-08-23 17:11:04+00:00
4,Akita,2012-03-03 09:31:08+00:00,2019-09-20 15:19:57+00:00
...,...,...,...
211,Wirehaired Pointing Griffon,2016-06-29 20:03:55+00:00,2016-06-29 20:03:55+00:00
212,Wirehaired Terrier,2012-11-27 14:07:54+00:00,2019-09-19 22:52:45+00:00
213,Xoloitzcuintli / Mexican Hairless,2007-02-01 00:00:00+00:00,2019-09-08 11:15:54+00:00
214,Yellow Labrador Retriever,2010-05-31 00:00:00+00:00,2019-09-20 06:30:27+00:00


5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [103]:
male=dogs[dogs["sex"]=="Male"].groupby("contact_state",as_index=False).count()[["contact_state","id"]]
female=dogs[dogs["sex"]=="Female"].groupby("contact_state",as_index=False).count()[["contact_state","id"]]
unito=pd.merge(male,female,on="contact_state",suffixes=('_male', '_female'))
unito["difference"]=abs(unito["id_male"]-unito["id_female"])
unito[unito["difference"]==max(unito["difference"])]

,contact_state,id_male,id_female,difference
36,OH,1439,1234,205


6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [104]:
dogs.groupby(["age","size"]).mean()[["stay_duration","stay_cost"]]

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [105]:
involvedog=travels.groupby("id",as_index=False).count()[["id","index"]]
involvedog=involvedog[involvedog["index"]>=3]
pd.merge(involvedog,dogs,on="id")[["id","index","breed_primary","breed_secondary"]]

,id,index,breed_primary,breed_secondary
0,16657005,4,Pit Bull Terrier,NaN
1,20905974,5,Chow Chow,English Springer Spaniel
2,24894870,4,Hound,Terrier
3,24894894,4,Hound,Terrier
4,33218331,7,Alaskan Malamute,Siberian Husky
...,...,...,...,...
558,46042569,3,Labrador Retriever,NaN
559,46042587,3,Labrador Retriever,NaN
560,46042618,3,Labrador Retriever,NaN
561,46043099,3,Labrador Retriever,NaN


8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

In [106]:
lista=[]
for i in range(len(travels)):
    if pd.isnull(travels["manual"][i])==False:
        lista.append(travels["manual"][i])
    else:
        lista.append(travels["found"][i])
travels["correct_state"]=lista
travels

,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN,Arkansas
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN,Bahamas
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN,Maryland
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN,Adaptil
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN,Afghanistan
...,...,...,...,...,...,...,...,...,...,...
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN,WV
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN,Wyoming
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN,Yazmin
6192,6192,44519341,Dayton,OH,Callie is a 14 year old Chihuahua whose owner ...,Young,Ohio,NaN,NaN,Ohio


9. For each state, compute the ratio between the number of travels and the population.

In the population dataset i have extended state name but in the travels dataset i have an abbreviation, so need an external dataset that tells me the abbreviation of each state

In [107]:
stateabbr=pd.read_csv("statelatlong.csv")
stateabbr

,State,Latitude,Longitude,City
0,AL,32.601011,-86.680736,Alabama
1,AK,61.302501,-158.775020,Alaska
2,AZ,34.168219,-111.930907,Arizona
3,AR,34.751928,-92.131378,Arkansas
4,CA,37.271875,-119.270415,California
5,CO,38.997934,-105.550567,Colorado
6,CT,41.518784,-72.757507,Connecticut
7,DE,39.145251,-75.418921,Delaware
8,DC,38.899349,-77.014567,District of Columbia
9,FL,27.975728,-83.833017,Florida


In [108]:
nst=pd.merge(nst,stateabbr,left_on="State",right_on="City",suffixes=('', '_abbr'))[["State","Population","State_abbr"]]
nst

,State,Population,State_abbr
0,Alabama,5024279,AL
1,Alaska,733391,AK
2,Arizona,7151502,AZ
3,Arkansas,3011524,AR
4,California,39538223,CA
5,Colorado,5773714,CO
6,Connecticut,3605944,CT
7,Delaware,989948,DE
8,District of Columbia,689545,DC
9,Florida,21538187,FL


In [109]:
travels.groupby("contact_state").count()

,index,id,contact_city,description,found,manual,remove,still_there,correct_state
contact_state,,,,,,,,,
17325,10,10,10,10,10,0,10,0,10
AL,75,75,75,75,75,10,17,1,75
AR,10,10,10,10,10,1,3,2,10
AZ,70,70,70,70,70,18,37,1,70
CA,28,28,28,28,28,10,15,0,28
CO,103,103,103,103,103,45,11,1,103
CT,90,90,90,90,90,26,12,6,90
DC,112,112,112,112,112,50,61,7,112
DE,57,57,57,57,57,5,25,0,57


We can notice a 17325 as contact_state that it is not correct

In [110]:
travels[travels["contact_state"]=="17325"]

,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state
3237,3237,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,New York,NaN,True,NaN,New York
3238,3238,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",New York,NaN,True,NaN,New York
3714,3714,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Pennsylvania,NaN,True,NaN,Pennsylvania
3715,3715,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Pennsylvania,NaN,True,NaN,Pennsylvania
6029,6029,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Virginia,NaN,True,NaN,Virginia
6030,6030,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Virginia,NaN,True,NaN,Virginia
6074,6074,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Washington DC,NaN,True,NaN,Washington DC
6075,6075,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Washington DC,NaN,True,NaN,Washington DC
6133,6133,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,West Virginia,NaN,True,NaN,West Virginia
6134,6134,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",West Virginia,NaN,True,NaN,West Virginia


They have all contact_city as PA that it is not a city

In [111]:
travels[travels["contact_city"]=="PA"]

,index,id,contact_city,contact_state,description,found,manual,remove,still_there,correct_state
2472,2472,36978896,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,Maryland,NaN,True,NaN,Maryland
2473,2473,33218331,PA,PA,"Born in August 2014, Bucky has a great sense o...",Maryland,NaN,True,NaN,Maryland
3190,3190,36978896,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,New Jersey,NaN,True,NaN,New Jersey
3191,3191,33218331,PA,PA,"Born in August 2014, Bucky has a great sense o...",New Jersey,NaN,True,NaN,New Jersey
3237,3237,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,New York,NaN,True,NaN,New York
3238,3238,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",New York,NaN,True,NaN,New York
3714,3714,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Pennsylvania,NaN,True,NaN,Pennsylvania
3715,3715,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Pennsylvania,NaN,True,NaN,Pennsylvania
6029,6029,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Virginia,NaN,True,NaN,Virginia
6030,6030,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Virginia,NaN,True,NaN,Virginia


We try to join with the dataset in order to infere to the real contact_state and contact_city

In [112]:
pd.merge(dogs[["id","contact_zip","contact_state","contact_city"]],travels[travels["contact_city"]=="PA"],on="id")

,id,contact_zip,contact_state_x,contact_city_x,index,contact_city_y,contact_state_y,description,found,manual,remove,still_there,correct_state
0,36978896,17325,PA,Gettysburg,2472,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,Maryland,NaN,True,NaN,Maryland
1,36978896,17325,PA,Gettysburg,3190,PA,PA,Maddie is our little Miss Cutie Patootie! She ...,New Jersey,NaN,True,NaN,New Jersey
2,36978896,17325,PA,Gettysburg,3237,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,New York,NaN,True,NaN,New York
3,36978896,17325,PA,Gettysburg,3714,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Pennsylvania,NaN,True,NaN,Pennsylvania
4,36978896,17325,PA,Gettysburg,6029,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Virginia,NaN,True,NaN,Virginia
5,36978896,17325,PA,Gettysburg,6074,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Washington DC,NaN,True,NaN,Washington DC
6,36978896,17325,PA,Gettysburg,6133,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,West Virginia,NaN,True,NaN,West Virginia
7,33218331,17325,PA,Gettysburg,2473,PA,PA,"Born in August 2014, Bucky has a great sense o...",Maryland,NaN,True,NaN,Maryland
8,33218331,17325,PA,Gettysburg,3191,PA,PA,"Born in August 2014, Bucky has a great sense o...",New Jersey,NaN,True,NaN,New Jersey
9,33218331,17325,PA,Gettysburg,3238,PA,17325,"Born in August 2014, Bucky has a great sense o...",New York,NaN,True,NaN,New York


In [113]:
travels["contact_city"]=travels["contact_city"].replace("PA","Gettysburg")
travels["contact_state"]=travels["contact_state"].replace("17325","PA")

Now we can solve the problem  

In [114]:
finalratio=pd.merge(nst,travels.groupby("contact_state",as_index=False).count()[["contact_state","id"]],left_on="State_abbr",right_on="contact_state",how="left").fillna(0)
finalratio["ratio"]=finalratio["id"]/finalratio["Population"]
finalratio[["State","ratio"]].sort_values(by="ratio",ascending=False)

,State,ratio
19,Maine,2.649816e-04
8,District of Columbia,1.624259e-04
46,Virginia,1.187526e-04
39,Rhode Island,1.184641e-04
47,Washington,8.228123e-05
45,Vermont,7.619616e-05
20,Maryland,6.135442e-05
30,New Jersey,5.942516e-05
7,Delaware,5.757878e-05
31,New Mexico,3.872451e-05


10. For each dog, compute the number of days from the posted day to the day of last access.

In [115]:
dogs.head()[["posted","accessed"]]

,posted,accessed
0,2019-09-20 16:37:59+00:00,2019-09-20
1,2019-09-20 16:24:57+00:00,2019-09-20
2,2019-09-20 14:10:11+00:00,2019-09-20
3,2019-09-20 10:08:22+00:00,2019-09-20
4,2019-09-20 06:48:30+00:00,2019-09-20


To take only the day

In [116]:
dogs["posteddate"]=pd.to_datetime(dogs["posted"].dt.date)

In [117]:
dogs["diffday"]=dogs["accessed"]-dogs["posteddate"]
dogs["diffday"]

0         0 days
1         0 days
2         0 days
3         0 days
4         0 days
          ...   
58175   140 days
58176   160 days
58177   358 days
58178   373 days
58179   382 days
Name: diffday, Length: 58180, dtype: timedelta64[ns]

In [118]:
dogs.loc[:,("id","diffday")]

,id,diffday
0,46042150,0 days
1,46042002,0 days
2,46040898,0 days
3,46039877,0 days
4,46039306,0 days
...,...,...
58175,44605893,140 days
58176,44457061,160 days
58177,42865848,358 days
58178,42734734,373 days


11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

In [119]:
delta=dogs["diffday"]
dogs["number_of_weeks"]=delta.dt.days//7
dogs["number_of_weeks"]

0         0
1         0
2         0
3         0
4         0
         ..
58175    20
58176    22
58177    51
58178    53
58179    54
Name: number_of_weeks, Length: 58180, dtype: int64

In [120]:
dogs.groupby("number_of_weeks",as_index=False).count()[["number_of_weeks","id"]]

,number_of_weeks,id
0,0,9803
1,1,6547
2,2,5764
3,3,3353
4,4,2439
...,...,...
574,729,1
575,746,1
576,811,1
577,812,1


12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

I must first divide the case where the description is present and the case where it is not. We will not consider the case where there is no description

In [189]:
dogs2=dogs[dogs["description"].notnull()]
dogs2

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,posteddate,diffday,number_of_weeks
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,US,NV,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81,2019-09-20,0 days,0
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,US,NV,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07,2019-09-20,0 days,0
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,US,NV,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51,2019-09-20,0 days,0
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,US,NV,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09,2019-09-20,0 days,0
5,46039304,NV184,https://www.petfinder.com/dog/oliver-46039304/...,Dog,Dog,Boxer,Beagle,True,False,NaN,...,US,NV,2019-09-20,Dog,Oliver was born around mid-June and came to us...,70,97.77,2019-09-20,0 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58166,45168741,WY24,https://www.petfinder.com/dog/charlie-45168741...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,US,WY,2019-09-20,Dog,"Charlie was adopted from us 5 years ago, but r...",109,280.51,2019-07-04,78 days,11
58167,44843897,WY24,https://www.petfinder.com/dog/samson-44843897/...,Dog,Dog,Pit Bull Terrier,NaN,False,False,NaN,...,US,WY,2019-09-20,Dog,Samson is a dog that will need someone to show...,74,132.32,2019-05-31,112 days,16
58172,44658860,WY24,https://www.petfinder.com/dog/buddy-44658860/w...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,NaN,...,US,WY,2019-09-20,Dog,Buddy was an owner surrender by an older gentl...,135,357.97,2019-05-10,133 days,19
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",...,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,324.34,2019-05-03,140 days,20


In [190]:
newdescription=[]
from unicodedata import category, normalize
for nome in dogs2["description"]:
    newdescription.append("".join(c for c in normalize('NFD',nome) if category(c) != 'Mn').lower())
dogs2["description"]=newdescription

C:\Users\marta\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [191]:
dogs2["description"]=dogs2["description"].str.replace("\n"," ")

C:\Users\marta\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [192]:
import nltk
from nltk.corpus import stopwords
stopword=stopwords.words('english')
stopword

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [193]:
filtered_sentences = []
for frase in dogs2["description"]:
    sen=""
    for elem in frase.split():
        if elem not in stopword:
            sen=sen+" "+elem
    filtered_sentences.append(sen)

In [194]:
dogs2["description"]=filtered_sentences

C:\Users\marta\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [195]:
def jaccard_similarity(x,y):
  intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
  union_cardinality = len(set.union(*[set(x), set(y)]))
  return intersection_cardinality/float(union_cardinality)

Case number one: a secondary breed is present 

In [196]:
combinazioni=dogs2.groupby(['breed_primary', 'breed_secondary', 'sex'],as_index=False).count()[['breed_primary', 'breed_secondary', 'sex']]
combinazioni

,breed_primary,breed_secondary,sex
0,Affenpinscher,Chihuahua,Male
1,Affenpinscher,Mixed Breed,Male
2,Afghan Hound,Cocker Spaniel,Male
3,Airedale Terrier,Catahoula Leopard Dog,Male
4,Airedale Terrier,Coonhound,Female
...,...,...,...
3905,Yorkshire Terrier,Silky Terrier,Female
3906,Yorkshire Terrier,Silky Terrier,Male
3907,Yorkshire Terrier,Terrier,Female
3908,Yorkshire Terrier,Terrier,Male


In [197]:
dogs3=dogs2[dogs2["breed_secondary"].notnull()]
dogs3

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,posteddate,diffday,number_of_weeks
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,US,NV,2019-09-20,Dog,harley sure wound shelter senior years see pi...,70,124.81,2019-09-20,0 days,0
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,US,NV,2019-09-20,Dog,6 year old biggie lost home really wants home...,49,122.07,2019-09-20,0 days,0
5,46039304,NV184,https://www.petfinder.com/dog/oliver-46039304/...,Dog,Dog,Boxer,Beagle,True,False,NaN,...,US,NV,2019-09-20,Dog,oliver born around mid-june came us recently ...,70,97.77,2019-09-20,0 days,0
6,46039303,NV184,https://www.petfinder.com/dog/macadamia-460393...,Dog,Dog,Italian Greyhound,Chihuahua,True,False,NaN,...,US,NV,2019-09-20,Dog,macadamia born around july 8th dona¢a€a™t let...,80,103.53,2019-09-20,0 days,0
10,46038708,NV184,https://www.petfinder.com/dog/speckles-4603870...,Dog,Dog,Cattle Dog,Hound,True,False,NaN,...,US,NV,2019-09-20,Dog,speckles 4 month old heeler/hound mix gentle ...,84,320.37,2019-09-20,0 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58163,45168743,WY24,https://www.petfinder.com/dog/sammy-45168743/w...,Dog,Dog,Labrador Retriever,Mixed Breed,True,False,NaN,...,US,WY,2019-09-20,Dog,sammy one happy puppy literally wrangle get p...,115,290.28,2019-07-04,78 days,11
58164,45168750,WY24,https://www.petfinder.com/dog/dwayne-45168750/...,Dog,Dog,Australian Cattle Dog / Blue Heeler,Mixed Breed,True,False,NaN,...,US,WY,2019-09-20,Dog,"dwayne typical adolescent: happy, playful, fu...",110,254.27,2019-07-04,78 days,11
58165,45168752,WY24,https://www.petfinder.com/dog/sebastian-451687...,Dog,Dog,Australian Shepherd,Border Collie,True,False,NaN,...,US,WY,2019-09-20,Dog,sebastian typical herding dog: high energy ne...,105,171.81,2019-07-04,78 days,11
58166,45168741,WY24,https://www.petfinder.com/dog/charlie-45168741...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,US,WY,2019-09-20,Dog,"charlie adopted us 5 years ago, recently came...",109,280.51,2019-07-04,78 days,11


In [198]:
idduplicati=[]
for i in range(len(combinazioni)):
    intermedio=dogs3[(dogs3["breed_primary"]==combinazioni.breed_primary.iloc[i]) & (dogs3["breed_secondary"]==combinazioni.breed_secondary.iloc[i]) & (dogs3["sex"]==combinazioni.sex.iloc[i])]
    
    if len(intermedio)>1:
        for val in range(len(intermedio)):
            for k in range(len(intermedio)):
                if val!=k:
                    lista1=intermedio.iloc[val]["description"].split()
                    lista2=intermedio.iloc[k]["description"].split()
                    if jaccard_similarity(lista1,lista2)>=0.9:
                        idduplicati.append([intermedio.iloc[val].id,intermedio.iloc[k].id])
                

In [199]:
len(idduplicati)

3116

In [200]:
idduplicati

[[45733110, 45732409],
 [45732409, 45733110],
 [45932298, 45931729],
 [45931729, 45932298],
 [45932359, 45932317],
 [45932359, 45932138],
 [45932359, 45932104],
 [45932359, 45931760],
 [45932359, 45931720],
 [45932359, 45931696],
 [45932359, 45931670],
 [45932359, 45931656],
 [45932359, 45931611],
 [45932359, 45931579],
 [45932359, 45931524],
 [45932317, 45932359],
 [45932317, 45932138],
 [45932317, 45932104],
 [45932317, 45931760],
 [45932317, 45931720],
 [45932317, 45931696],
 [45932317, 45931670],
 [45932317, 45931656],
 [45932317, 45931611],
 [45932317, 45931579],
 [45932317, 45931524],
 [45932138, 45932359],
 [45932138, 45932317],
 [45932138, 45932104],
 [45932138, 45931760],
 [45932138, 45931720],
 [45932138, 45931696],
 [45932138, 45931670],
 [45932138, 45931656],
 [45932138, 45931611],
 [45932138, 45931579],
 [45932138, 45931524],
 [45932104, 45932359],
 [45932104, 45932317],
 [45932104, 45932138],
 [45932104, 45931760],
 [45932104, 45931720],
 [45932104, 45931696],
 [45932104,

Case number two: there is no secondary breed 

In [202]:
dogs4=dogs2[dogs2["breed_secondary"].isnull()]
dogs4

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,posteddate,diffday,number_of_weeks
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,US,NV,2019-09-20,Dog,approx 2 years old. catch eye? blame stop sta...,87,281.51,2019-09-20,0 days,0
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,US,NV,2019-09-20,Dog,theo friendly dachshund mix gets along well d...,93,241.09,2019-09-20,0 days,0
7,46039302,NV184,https://www.petfinder.com/dog/dodger-46039302/...,Dog,Dog,Cattle Dog,NaN,True,False,NaN,...,US,NV,2019-09-20,Dog,"dodger handsome, smart heeler mix born may 26...",46,79.91,2019-09-20,0 days,0
8,46039301,NV184,https://www.petfinder.com/dog/huckleberry-4603...,Dog,Dog,Cattle Dog,NaN,True,False,NaN,...,US,NV,2019-09-20,Dog,huckleberry friendly girl born may 26th. full...,97,252.17,2019-09-20,0 days,0
9,46038709,NV184,https://www.petfinder.com/dog/fagin-46038709/n...,Dog,Dog,Cattle Dog,NaN,True,False,NaN,...,US,NV,2019-09-20,Dog,"fagin born may 26th smart, inquisitive boy ex...",112,137.19,2019-09-20,0 days,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58149,45813872,WY01,https://www.petfinder.com/dog/phoebe-45813872/...,Dog,Dog,Labrador Retriever,NaN,True,False,NaN,...,US,WY,2019-09-20,Dog,please stop day noon 2pm call 307-330-5200 in...,93,132.45,2019-08-31,20 days,2
58152,45734218,WY24,https://www.petfinder.com/dog/charlie-45734218...,Dog,Dog,Pit Bull Terrier,NaN,False,False,NaN,...,US,WY,2019-09-20,Dog,charlie surrendered back us kept digging fenc...,120,438.57,2019-08-24,27 days,3
58167,44843897,WY24,https://www.petfinder.com/dog/samson-44843897/...,Dog,Dog,Pit Bull Terrier,NaN,False,False,NaN,...,US,WY,2019-09-20,Dog,samson dog need someone show love is. soon br...,74,132.32,2019-05-31,112 days,16
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",...,US,WY,2019-09-20,Dog,"due small size volunteer base, unable reply i...",100,324.34,2019-05-03,140 days,20


In [203]:
combinazioni2=dogs4.groupby(["breed_primary","sex"],as_index=False).count()[["breed_primary","sex"]]
combinazioni2

,breed_primary,sex
0,Affenpinscher,Female
1,Affenpinscher,Male
2,Afghan Hound,Female
3,Afghan Hound,Male
4,Airedale Terrier,Female
...,...,...
367,Xoloitzcuintli / Mexican Hairless,Male
368,Yellow Labrador Retriever,Female
369,Yellow Labrador Retriever,Male
370,Yorkshire Terrier,Female


In [ ]:
idduplicati2=[]
for i in range(len(combinazioni2)):
    intermedio=dogs4[(dogs4["breed_primary"]==combinazioni2.breed_primary.iloc[i]) & (dogs4["sex"]==combinazioni2.sex.iloc[i])]
    
    if len(intermedio)>1:
        for val in range(len(intermedio)):
            for k in range(len(intermedio)):
                if val!=k:
                    lista1=intermedio.iloc[val]["description"].split()
                    lista2=intermedio.iloc[k]["description"].split()
                    if jaccard_similarity(lista1,lista2)>=0.9:
                        idduplicati2.append([intermedio.iloc[val].id,intermedio.iloc[k].id])

In [ ]:
idduplicati2

algoritmo se due record sono duplicati

In [149]:
dogs2=dogs2.set_index("id")

In [161]:
def duplicati(id1,id2):
    if dogs2.loc[id1]["breed_primary"]==dogs2.loc[id2]["breed_primary"] and dogs2.loc[id1]["breed_secondary"]==dogs2.loc[id2]["breed_secondary"] and dogs2.loc[id1]["sex"]==dogs2.loc[id2]["sex"]:
        lista1=dogs2.loc[id1]["description"].split()
        lista2=dogs2.loc[id2]["description"].split()
        if jaccard_similarity(lista1,lista2)>0.9:
            print("Duplicato")
        else:
            print("No Duplicato")
    else:
        print("No Duplicato")
        

In [163]:
duplicati(46042002,46042150)

No Duplicato


In [164]:
duplicati(45732409,45733110)

Duplicato
